# EXTRACCIÓN DE PLANTILLAS ACB - PROBALLERS

Este notebook extrae información de jugadores de los equipos ACB que participan en competiciones europeas. Aunque existe algún jugador que ya no juega porque ha cambiado de equipo pero que filtraremos posteriormente.

**Objetivo:** Generar un CSV con los links  de los jugadores para su posterior análisis y evitar extraerlo cada vez.

**Fuente:** [Proballers.com](https://www.proballers.com)

## 1. Imports 

In [19]:
import requests
from bs4 import BeautifulSoup
import polars as pl
from urllib.parse import urljoin
import re
import os
from datetime import datetime


## 2. Configuración

In [20]:
# URLs
BASE_URL = "https://www.proballers.com"
EQUIPOS_ACB_URL = f"{BASE_URL}/es/baloncesto/liga/30/spain-liga-endesa/equipos"

# Equipos ACB en competiciones europeas
EQUIPOS_EUROPEOS = [
    "Real Madrid", "Barcelona", "Valencia", "Baskonia", 
    "Unicaja", "Joventut", "Gran Canaria", "Manresa", 
    "Tenerife", "Bilbao", "Zaragoza"
]

# Rutas de salida
OUTPUT_DIR = "../data"
OUTPUT_FILE = "plantillas.csv"

## 3. Funciones

In [ ]:
def get_proballers_soup(url):
    session = requests.Session()
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127 Safari/537.36",
        "Referer": BASE_URL,  # ← Usa constante
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "es-ES,es;q=0.9",
        "Upgrade-Insecure-Requests": "1"
    }
    
    # Warm-up: primera visita para obtener cookies
    session.get(BASE_URL, headers=headers)  
    
    # Obtiene la página solicitada
    res = session.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    
    return soup


def get_plantillas(team_url):
    soup_team = get_proballers_soup(team_url)
    jugadores = soup_team.select(".select-tab-average-30 a")
    
    return pl.DataFrame({
        "nombres": [a.get_text(strip=True) for a in jugadores],
        "jugador_link": [urljoin(team_url, a.get('href')) for a in jugadores],
        "player_id": [re.search(r'\d+', a.get('href')).group() for a in jugadores]
    })

# 4. Extracción

In [ ]:
# Equipos ACB
soup = get_proballers_soup(EQUIPOS_ACB_URL)  # ← Usa constante
equipos = soup.select("a.home-league__team-list__content__entry-team__presentation")

# Filtra equipos europeos
df_equipos = pl.DataFrame({
    "nombres": [a.find('h3').get_text(strip=True) for a in equipos],
    "enlaces_equipo": [urljoin(EQUIPOS_ACB_URL, a.get('href')) for a in equipos],  
    "team_id": [re.search(r'\d+', a.get('href')).group() for a in equipos]
}).filter(
    pl.col("nombres").str.contains("|".join(EQUIPOS_EUROPEOS))  

# Extrae plantillas
team_urls = df_equipos['enlaces_equipo'].to_list()
df_todas_plantillas = pl.concat([get_plantillas(url) for url in team_urls])

# Guarda CSV
os.makedirs(OUTPUT_DIR, exist_ok=True)  # ← Usa constante
df_todas_plantillas.write_csv(f"{OUTPUT_DIR}/{OUTPUT_FILE}") 

df_todas_plantillas

nombres,jugador_link,player_id
str,str,str
"""Luka Samanic""","""https://www.proballers.com/es/…","""70813"""
"""Timothe Luwawu-Cabarrot""","""https://www.proballers.com/es/…","""61704"""
"""Trent Forrest""","""https://www.proballers.com/es/…","""72819"""
"""Hamidou Diallo""","""https://www.proballers.com/es/…","""74039"""
"""Kobi Simmons""","""https://www.proballers.com/es/…","""72846"""
…,…,…
"""Neal Sako""","""https://www.proballers.com/es/…","""71342"""
"""Joseph Puerto""","""https://www.proballers.com/es/…","""67766"""
"""Yankuba Sima""","""https://www.proballers.com/es/…","""65333"""
